<a href="https://colab.research.google.com/github/fernandofsilva/LSTM_Option_Pricing/blob/main/notebooks/03_modelo_dnn_subclassing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [16]:
#@title Carregando as bibliotecas base
import pandas as pd
import numpy as np
import tensorflow as tf

import warnings
warnings.simplefilter('ignore')

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn

seaborn.set_style('whitegrid')

In [17]:
#@title Carregando os dados
data = pd.read_csv(f'/content/drive/My Drive/Mestrado/data/dados_treino_teste.csv.gz', compression='gzip', index_col=0)
data.head()

,codigo,mercado,preco_opcao,preco_exercicio,data_vencimento,T,preco_ativo,volatilidade,taxa_juros,black_scholes,delta_black_scholes,base
2015-01-05,PETRM17,OPÇÕES DE VENDA,8.18,16.91,2015-01-19,0.039683,8.61,0.771953,0.1157,8.22,100,treino
2015-01-05,PETRM28,OPÇÕES DE VENDA,0.36,8.41,2015-01-19,0.039683,8.61,0.771953,0.1157,0.41,40,treino
2015-01-05,PETRM2,OPÇÕES DE VENDA,0.02,4.91,2015-01-19,0.039683,8.61,0.771953,0.1157,0.00,0,teste
2015-01-05,PETRM23,OPÇÕES DE VENDA,0.79,9.21,2015-01-19,0.039683,8.61,0.771953,0.1157,0.87,63,treino
2015-01-05,PETRM25,OPÇÕES DE VENDA,1.13,9.61,2015-01-19,0.039683,8.61,0.771953,0.1157,1.16,73,treino


# Engenharia das variáveis (Feature Engineering)

Essa sessão é composta da transformação dos dados para entrada na rede na rede neural. Portando, as variáveis são transformadas do seu valor original, seja para adequação dentro da rede neural ou para um melhor treinamento da rede, essas transformações são:

- Variavéis númericas: preco_exercicio, preco_ativo, foram normalizadas antes da entrada na rede
- Variavéis númericas: preco_opcao (alvo), volatilidade, taxa_juros e T não sofreram alterações
- Variável categórica mercado sofreu one hot encoding

A transformação dos dados é feita no mesmo momento que o modelo é treinado, isso é feito através de uma camada dentro do modelo, essa camada tem o nome de feature layer.

In [18]:
#@title Pipeline de entrada dos dados
def df_to_dataset(dataframe, base, shuffle=True, batch_size=22):

    # Criar cópia do dataframe
    dataframe = dataframe.copy()

    # Filtrar a base
    dataframe = dataframe[dataframe['base'] == base]

    # Variavel alvo
    labels = dataframe.pop('preco_opcao')

    # Colunas do modelo
    cols = ['mercado', 'preco_exercicio', 'preco_ativo', 'T', 'volatilidade', 'taxa_juros']

    # Criar o td.data
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe[cols]), labels))

    # Embaralhar os dados se necessário
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))

    # Criar o batch de dados
    ds = ds.batch(batch_size)

    return ds

# Divisão da base de treino e teste
train_ds = df_to_dataset(data, base='treino')
test_ds = df_to_dataset(data, shuffle=False, base='teste')

In [19]:
#@title Mapeamento das colunas
feature_columns = []

# Colunas númericas normalizadas
for column in ['preco_exercicio', 'preco_ativo']:
    
    mean = data.loc[data['base'] == 'treino', column].mean()
    stdev = data.loc[data['base'] == 'treino', column].std()

    feature_columns.append(tf.feature_column.numeric_column(column, normalizer_fn = lambda x: (x - mean) / stdev))

# Colunas númericas sem normalização
for column in ['T', 'volatilidade', 'taxa_juros']:

    feature_columns.append(tf.feature_column.numeric_column(column))

# Colunas categóricas
option = tf.feature_column.categorical_column_with_vocabulary_list('mercado', ['OPÇÕES DE COMPRA', 'OPÇÕES DE VENDA'])
option_one_hot = tf.feature_column.indicator_column(option)
feature_columns.append(option_one_hot)

In [20]:
#@title Camada de transformação (feature layer)
feature_layer = tf.keras.layers.DenseFeatures(feature_columns, name='Feature')

# Modelo

O modelo de rede neural profunda a seguir, foi baseado nos estudos desenvolvidos por Hirsa, Karatas, & Oskoui. No trabalho são testadas diversas arquiteturas (camadas e elementos em cada camada), bem como função de atição de cada camada e também função de otimização.

A conclusão do estudo mostra que os melhores resultados foram obtidos utilizando uma rede de 4 camadas com 120 neurônios cada uma.

In [21]:
#@title Model Subclassing
class DNN_Model(tf.keras.Model):
    def __init__(self):
        super(DNN_Model, self).__init__()
        self.feature = feature_layer
        self.d1 = tf.keras.layers.Dense(120, activation='relu', name='Dense_1')
        self.d2 = tf.keras.layers.Dense(120, activation='relu', name='Dense_2')
        self.d3 = tf.keras.layers.Dense(120, activation='relu', name='Dense_3')
        self.d4 = tf.keras.layers.Dense(120, activation='relu', name='Dense_4')
        self.output_kayer = tf.keras.layers.Dense(1, name='output')

    def call(self, inputs):
        h = self.feature(inputs)
        h = self.d1(h)
        h = self.d2(h)
        h = self.d3(h)
        h = self.d4(h)
        return self.output_kayer(h)

# Create an instance of the model
model = DNN_Model()

In [22]:
#@title Inicialização do modelo
# Para apresentar o sumário do modelo, primeiramente, é necessário inicializar os pesos e bias de cada camada,
# isso é feito utilizando uma amostra dos dados
element = next(iter(train_ds.take(1)))[0]
model(element)

<tf.Tensor: shape=(22, 1), dtype=float32, numpy=
array([[ 0.04783595],
       [ 0.07835528],
       [ 0.00552963],
       [ 0.00041123],
       [ 0.07175818],
       [ 0.06814009],
       [ 0.14946648],
       [ 0.13198915],
       [ 0.0051135 ],
       [ 0.05440015],
       [ 0.10923524],
       [ 0.0741863 ],
       [ 0.05338414],
       [ 0.04139257],
       [ 0.06089517],
       [ 0.13722904],
       [ 0.02693585],
       [ 0.05808833],
       [ 0.02142004],
       [-0.00056502],
       [ 0.07546006],
       [ 0.04938966]], dtype=float32)>

In [23]:
#@title Sumário do modelo
model.summary()

Model: "dnn__model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Feature (DenseFeatures)      multiple                  0         
_________________________________________________________________
Dense_1 (Dense)              multiple                  960       
_________________________________________________________________
Dense_2 (Dense)              multiple                  14520     
_________________________________________________________________
Dense_3 (Dense)              multiple                  14520     
_________________________________________________________________
Dense_4 (Dense)              multiple                  14520     
_________________________________________________________________
output (Dense)               multiple                  121       
Total params: 44,641
Trainable params: 44,641
Non-trainable params: 0
__________________________________________________

In [24]:
#@title Função de perda e otimizador
loss_object = tf.keras.losses.MeanSquaredError(reduction="auto", name="mse")
optimizer = tf.keras.optimizers.Adam(0.001)

In [25]:
#@title Métricas
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_mae = tf.keras.metrics.MeanAbsoluteError(name='train_mae')
train_mape = tf.keras.metrics.MeanAbsolutePercentageError(name='train_mape')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_mae = tf.keras.metrics.MeanAbsoluteError(name='test_mae')
test_mape = tf.keras.metrics.MeanAbsolutePercentageError(name='test_mape')

In [26]:
#@title Aplicando gradiente
@tf.function
def train_step(data, labels):
    with tf.GradientTape() as tape:
        predictions = model(data, training=True)
        loss = loss_object(predictions, labels)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_mae(labels, predictions)
    train_mape(labels, predictions)

In [27]:
#@title Função de teste
@tf.function
def test_step(data, labels):
  predictions = model(data, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_mae(labels, predictions)
  test_mape(labels, predictions)

In [28]:
#@title Treinamento do modelo
EPOCHS = 5

for epoch in range(EPOCHS):
    # Re-inicia as métricas a cada época
    train_loss.reset_states()
    train_mae.reset_states()
    train_mape.reset_states()
    test_loss.reset_states()
    test_mae.reset_states()
    test_mape.reset_states()

    for data, labels in train_ds:
        train_step(data, labels)

    for test_data, test_labels in test_ds:
        test_step(test_data, test_labels)

    print(
        f'Epoch {epoch + 1}, '
        f'Loss: {train_loss.result():.4f}, '
        f'MAE: {train_mae.result():.4f}, '
        f'MAPE: {train_mape.result():.4f}, '
        f'Test_Loss: {test_loss.result():.4f}, '
        f'Test_MAE: {test_mae.result():.4f}, '
        f'Test_MAPE: {test_mape.result():.4f}'
        )

Epoch 1, Loss: 0.1194, MAE: 0.1829, MAPE: 101.0916, Test_Loss: 0.0793, Test_MAE: 0.1432, Test_MAPE: 47.8851
Epoch 2, Loss: 0.0643, MAE: 0.1444, MAPE: 56.6683, Test_Loss: 0.0730, Test_MAE: 0.1517, Test_MAPE: 82.7700
Epoch 3, Loss: 0.0595, MAE: 0.1359, MAPE: 49.4900, Test_Loss: 0.0604, Test_MAE: 0.1247, Test_MAPE: 53.2899
Epoch 4, Loss: 0.0575, MAE: 0.1322, MAPE: 46.1591, Test_Loss: 0.0621, Test_MAE: 0.1363, Test_MAPE: 53.0687
Epoch 5, Loss: 0.0562, MAE: 0.1292, MAPE: 44.7188, Test_Loss: 0.0604, Test_MAE: 0.1334, Test_MAPE: 62.3024
